In [1]:
import nfl_data_py as nfl
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
import numpy as np
import os
import requests
import json

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [2]:
API_KEY = '658dc172e12821ff0c3ea93566e246c8'
REGIONS = 'us'
MARKETS = 'spreads'
ODDS_FORMAT = 'american'
DATE_FORMAT = 'iso'
SPORT = 'americanfootball_nfl'
KEY = 'betonlineag'
sports_response = requests.get(
    'https://api.the-odds-api.com/v4/sports', 
    params={
        'api_key': API_KEY
    }
)


In [3]:
if sports_response.status_code != 200:
    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')

else:
    print('List of in season sports:', sports_response.json())

List of in season sports: [{'key': 'americanfootball_cfl', 'group': 'American Football', 'title': 'CFL', 'description': 'Canadian Football League', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_ncaaf', 'group': 'American Football', 'title': 'NCAAF', 'description': 'US College Football', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_nfl', 'group': 'American Football', 'title': 'NFL', 'description': 'US Football', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_nfl_super_bowl_winner', 'group': 'American Football', 'title': 'NFL Super Bowl Winner', 'description': 'Super Bowl Winner 2022/2023', 'active': True, 'has_outrights': True}, {'key': 'aussierules_afl', 'group': 'Aussie Rules', 'title': 'AFL', 'description': 'Aussie Football', 'active': True, 'has_outrights': False}, {'key': 'baseball_mlb', 'group': 'Baseball', 'title': 'MLB', 'description': 'Major League Baseball', 'active': True, 'has_outrights': False}, {'key': 'basebal

In [4]:
odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds', params={
    'api_key': API_KEY,
    'regions': REGIONS,
    'markets': MARKETS,
    'oddsFormat': ODDS_FORMAT,
    'dateFormat': DATE_FORMAT,
    'bookmakers': KEY
    
})

In [5]:
odds_response.json()

[{'id': '44991d2ca9683719269485ff8a7143db',
  'sport_key': 'americanfootball_nfl',
  'sport_title': 'NFL',
  'commence_time': '2022-09-23T00:15:00Z',
  'home_team': 'Cleveland Browns',
  'away_team': 'Pittsburgh Steelers',
  'bookmakers': [{'key': 'betonlineag',
    'title': 'BetOnline.ag',
    'last_update': '2022-09-20T21:20:53Z',
    'markets': [{'key': 'spreads',
      'outcomes': [{'name': 'Cleveland Browns', 'price': -111, 'point': -4.5},
       {'name': 'Pittsburgh Steelers', 'price': -111, 'point': 4.5}]}]}]},
 {'id': '17992ef53890f217f965a6918e65c5d2',
  'sport_key': 'americanfootball_nfl',
  'sport_title': 'NFL',
  'commence_time': '2022-09-25T17:00:00Z',
  'home_team': 'New England Patriots',
  'away_team': 'Baltimore Ravens',
  'bookmakers': [{'key': 'betonlineag',
    'title': 'BetOnline.ag',
    'last_update': '2022-09-20T21:20:53Z',
    'markets': [{'key': 'spreads',
      'outcomes': [{'name': 'Baltimore Ravens', 'price': -115, 'point': -3.0},
       {'name': 'New Engla

In [6]:
spreads = pd.json_normalize(odds_response.json(), record_path=['bookmakers', 'markets', 'outcomes'], errors='ignore')
spreads = spreads.drop(spreads[spreads['point']>0].index)
spreads = spreads.reset_index(drop=True)


In [7]:
spreads

,name,price,point
0,Cleveland Browns,-111,-4.5
1,Baltimore Ravens,-115,-3.0
2,Buffalo Bills,-111,-5.5
3,New Orleans Saints,-104,-3.0
4,Chicago Bears,-119,-2.5
5,Cincinnati Bengals,-115,-4.5
6,Minnesota Vikings,-109,-6.0
7,Kansas City Chiefs,-105,-6.5
8,Las Vegas Raiders,-105,-2.5
9,Philadelphia Eagles,-102,-7.0


In [8]:
games = pd.json_normalize(odds_response.json())

frames = [games, spreads]
result = pd.concat([games,spreads], axis=1,)
result = result.drop(['id','sport_key','bookmakers'], axis=1)
result

,sport_title,commence_time,home_team,away_team,name,price,point
0,NFL,2022-09-23T00:15:00Z,Cleveland Browns,Pittsburgh Steelers,Cleveland Browns,-111,-4.5
1,NFL,2022-09-25T17:00:00Z,New England Patriots,Baltimore Ravens,Baltimore Ravens,-115,-3.0
2,NFL,2022-09-25T17:00:00Z,Miami Dolphins,Buffalo Bills,Buffalo Bills,-111,-5.5
3,NFL,2022-09-25T17:00:00Z,Carolina Panthers,New Orleans Saints,New Orleans Saints,-104,-3.0
4,NFL,2022-09-25T17:00:00Z,Chicago Bears,Houston Texans,Chicago Bears,-119,-2.5
5,NFL,2022-09-25T17:00:00Z,New York Jets,Cincinnati Bengals,Cincinnati Bengals,-115,-4.5
6,NFL,2022-09-25T17:00:00Z,Minnesota Vikings,Detroit Lions,Minnesota Vikings,-109,-6.0
7,NFL,2022-09-25T17:00:00Z,Indianapolis Colts,Kansas City Chiefs,Kansas City Chiefs,-105,-6.5
8,NFL,2022-09-25T17:00:00Z,Tennessee Titans,Las Vegas Raiders,Las Vegas Raiders,-105,-2.5
9,NFL,2022-09-25T17:00:00Z,Washington Commanders,Philadelphia Eagles,Philadelphia Eagles,-102,-7.0
